In [6]:
from database.main import Database

In [7]:
db = Database()

In [10]:
data = db.get_user_data_for_verification("8630111400")

In [11]:
data

{'name': 'Aditya Garg',
 'town_city': 'Nanakmatta',
 'state': 'Uttarakhand',
 'pincode': 262311}